In [48]:
import sys, os
import sys, os
import json
import pandas as pd
import numpy as np
from IPython.display import Markdown, display
import ipywidgets as widgets
import plotly
import plotly.graph_objects as graph_obj
import plotly.figure_factory as ff

sys.path.append(os.path.join(os.getcwd(),'/app/utils'))
from s3 import s3store
from jsonutil import WriteDictJson, ReadDictJson

In [49]:
output = widgets.Output()
clear_output = widgets.Button(description='Clear Output')

test_select = widgets.Select(
    options=[],
    description='Test:',
    disabled=False,
    #rows=25,
)

In [50]:
creds = ReadDictJson('../creds.json')
if not creds:
    output.append_stdout('Failed to load credentials file {}\n'.format(args.credentails))

s3def = creds['s3'][0]
s3 = s3store(s3def['address'], s3def['access key'], s3def['secret key'])
validation = s3def['sets']['trainingset']

test_obj = '{}/coco/tests.json'.format(validation['prefix'])
desc_obj = '{}/coco/description.json'.format(validation['prefix'])
testjson = s3.GetDict(validation['bucket'], test_obj)
descjson = s3.GetDict(validation['bucket'], desc_obj)

tests = pd.DataFrame(testjson)
display(tests)

,date,model,accuracy,class_similarity,similarity,confusion,images,image time,batch size,test store,test bucket,test object
0,"12/10/2020, 09:19:41",2020-12-03-05-52-45-cocoseg,0.869580,"{'0': {'intersection': 0, 'union': 0, 'similar...",0.634320,"[[842651, 2005, 1752, 17084], [30039, 53381, 2...",5,1.310252,1,ipc.larson.myds.me:32000,mllib,training/coco/2020-12-03-05-52-45-cocoseg/run....
1,"12/10/2020, 09:32:32",2020-12-03-05-52-45-cocoseg,0.933269,"{'0': {'intersection': 0, 'union': 0, 'similar...",0.599355,"[[1026475511, 6568840, 787060, 2550103], [2865...",5000,0.082757,1,ipc.larson.myds.me:32000,mllib,training/coco/2020-12-03-05-52-45-cocoseg/run....
2,"12/21/2020, 07:10:08",2020-12-19-14-56-02-cocoseg,0.946089,"{'0': {'intersection': 0, 'union': 0, 'similar...",0.678490,"[[1019482273, 11226207, 1588771, 3789864], [17...",5000,0.084817,1,ipc.larson.myds.me:32000,mllib,training/coco/2020-12-19-14-56-02-cocoseg/run....


In [51]:
iClasses = {}
classNames = [''] * descjson['classes']['classes']
for clasdesc in descjson['classes']['objects']:
    if clasdesc['trainId'] not in iClasses.keys():
        classNames[clasdesc['trainId']] = clasdesc['category']
        if not (clasdesc['category'] == 'void'):
            iClasses[clasdesc['trainId']] = clasdesc

overview = {}
results = {}
confusion = {}
test_names = []
    
for test in testjson:
    name = '{} {}'.format(test['model'], test['date'])
    test_names.append(name)
    
    test_overview = {'images':test['images'],'accuracy':test['accuracy'],'similarity':test['similarity'], 'inference time':test['image time']}
    overview[name]=test_overview
    
    similarity = {}
    for key in iClasses:
        res = test['class_similarity'][str(key)]
        if res:
            similarity[iClasses[key]['category']] = test['class_similarity'][str(key)]['similarity']

    results[name] = {'similarity':similarity, 'confusion':test['confusion']}
    
display(pd.DataFrame(overview).T)

,images,accuracy,similarity,inference time
"2020-12-03-05-52-45-cocoseg 12/10/2020, 09:19:41",5.0,0.869580,0.634320,1.310252
"2020-12-03-05-52-45-cocoseg 12/10/2020, 09:32:32",5000.0,0.933269,0.599355,0.082757
"2020-12-19-14-56-02-cocoseg 12/21/2020, 07:10:08",5000.0,0.946089,0.678490,0.084817


In [52]:
c = np.array(results[test_names[0]]['confusion'])
norm_confusion = (c.T / c.astype(np.float).sum(axis=1)).T
norm_confusion = norm_confusion.round(decimals=3)

confusion_text = [[str(round(y, 3)) for y in x] for x in norm_confusion]

# set up figure 
confusion_plot = plotly.figure_factory.create_annotated_heatmap(norm_confusion, x=classNames, y=classNames, colorscale='plasma')

# add custom xaxis title
confusion_plot.add_annotation(dict(font=dict(color="black",size=16),
                   x=0.5,
                   y=-0.15,
                   showarrow=False,
                   text="Predicted value",
                   xref="paper",
                   yref="paper"))

# add custom yaxis title
confusion_plot.add_annotation(dict(font=dict(color="black",size=16),
                   x=-0.2,
                   y=0.5,
                   showarrow=False,
                   text="Real value",
                   textangle=-90,
                   xref="paper",
                   yref="paper"))


# adjust margins to make room for yaxis title
confusion_plot.update_layout(margin=dict(t=50, l=200))

# add colorbar
confusion_plot['data'][0]['showscale'] = True
confusion_plot.update_layout(yaxis_autorange="reversed")

confusion_display = graph_obj.FigureWidget(confusion_plot)


In [53]:
def SelectTest(change, select, display, testjson):
    global test_select
    global confusion_display
    global output
    global results
    global confusion_plot
    
    with confusion_display.batch_update():
        
        c = np.array(results[test_select.value]['confusion'])
        norm_confusion = (c.T / c.astype(np.float).sum(axis=1)).T
        confusion_text = [str(round(x, 3)) for x in norm_confusion.flatten()]
        confusion_display.data[0].z =  norm_confusion
        confusion_display.update_layout(yaxis_autorange="reversed")
        
        for i in range(len(confusion_text)):
            confusion_display.layout.annotations[i].text = confusion_text[i]
            
def ClearOutput(b):
    global test_select
    global confusion_display
    global output
    output.clear_output()

In [54]:

clear_output.on_click(lambda b: ClearOutput(b))


display_results = widgets.HBox([test_select, confusion_display])
attach = widgets.VBox([display_results, clear_output, output])

test_select.observe (lambda change:SelectTest(change, select=test_select, display=confusion_display, testjson=testjson), names="value")

if len(test_names) > 0:
    test_select.options = test_names
    test_select.value = test_names[0]

display(attach)